In [13]:

# Import relevant Python modules
import operator
import math
import random
import numpy
import sys

from typing import List

# Import DEAP modules
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [14]:
N = 4

inputs  = [None] * 2 ** 4 # empty list, will hold the inputs for the truth table
outputs = [None] * 2 ** 4 # also empty list, will hold outputs for the truth table

# Generating the truth table

for i in range(2**4):
    row = format(i, '04b') # bit representation
    
    outputs[i] = row.count('1')%2==0 #true if there's an even number of 1s
    
    inputs[i] = [None] * 4 # making an empty sub-list in inputs
    for j in range(N):
        inputs[i][j] = row[j]=="1" # copying the contents of row into inputs[i]
        
print(inputs)
print("")
print(outputs)

[[False, False, False, False], [False, False, False, True], [False, False, True, False], [False, False, True, True], [False, True, False, False], [False, True, False, True], [False, True, True, False], [False, True, True, True], [True, False, False, False], [True, False, False, True], [True, False, True, False], [True, False, True, True], [True, True, False, False], [True, True, False, True], [True, True, True, False], [True, True, True, True]]

[True, False, False, True, False, True, True, False, False, True, True, False, True, False, False, True]


In [15]:
s1 = format(4, '04b') #formats the number 4 as a byte
print(s1)
print(s1.count('1')) # counts the 1s in the string s1
s2 = format(15, '04b') #formats the number 15 as a byte
print(s2)
print(s2.count('1')) # counts the 1s in the string s2

0100
1
1111
4


In [16]:
# And now time to set up the primitive sets

def NAND(x, y):
    return not(x and y)

def NOR(x, y):
    return not(x or y)

#adf 1
adfset1 = gp.PrimitiveSet("ADF1", 2)
adfset1.addPrimitive(operator.and_, 2)
adfset1.addPrimitive(operator.or_, 2)
adfset1.addPrimitive(NAND, 2)
adfset1.addPrimitive(NOR, 2)

#adf 0
adfset0 = gp.PrimitiveSet("ADF0", 2)
adfset0.addPrimitive(operator.and_, 2)
adfset0.addPrimitive(operator.or_, 2)
adfset0.addPrimitive(NAND, 2)
adfset0.addPrimitive(NOR, 2)
adfset0.addADF(adfset1)

#main pset
pset = gp.PrimitiveSet("MAIN", 4)
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(NAND, 2)
pset.addPrimitive(NOR, 2)
pset.addADF(adfset1)
pset.addADF(adfset0)

psets = (pset, adfset0, adfset1)

# Maximization function: weights set to (+1.0,)
# If it was minimization, it would have been set to (-1.0,)
creator.create("FitnessMin", base.Fitness, weights=(+1.0,))
creator.create("Tree",gp.PrimitiveTree)

creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("adf_expr0", gp.genFull, pset=adfset0, min_=1, max_=2)
toolbox.register("adf_expr1", gp.genFull, pset=adfset1, min_=1, max_=2)
toolbox.register("main_expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=4)

toolbox.register("ADF0", tools.initIterate, creator.Tree, toolbox.adf_expr0)
toolbox.register("ADF1", tools.initIterate, creator.Tree, toolbox.adf_expr1)
toolbox.register("MAIN", tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN, toolbox.ADF0, toolbox.ADF1]

toolbox.register("individual", tools.initCycle, creator.Individual, func_cycle)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalSymbReg(individual):
    func = toolbox.compile(individual)
    return -sum(abs(func(*in_) - out) for in_, out in zip(inputs, outputs)),

toolbox.register("compile", gp.compileADF, psets=psets)
toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr", gp.genFull, min_=1, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr)

#decorators: can limit height of the generated tree
#toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
#toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

random.seed() #(1024)
ind = toolbox.individual()

pop = toolbox.population(n=4000)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("mdn", numpy.median)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

logbook = tools.Logbook()
logbook.header = "gen","evals","std", "min","avg","max"

CXPB, MUTPB, NGEN = 0.7, 0.0, 30

# evaluate entire population

for ind in pop:
    ind.fitness.values = toolbox.evaluate(ind)
    
hof.update(pop)

record = stats.compile(pop)
logbook.record(gen=0, evals=len(pop), **record)
print(logbook.stream)

for g in range(1, NGEN):
    #select offspring
    offspring = toolbox.select(pop, len(pop))
    # clone them
    offspring = [toolbox.clone(ind) for ind in offspring]
    
    # apply crossover
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        for tree1, tree2 in zip(ind1, ind2):
            if random.random() < CXPB:
                toolbox.mate(tree1, tree2)
                del ind1.fitness.values
                del ind2.fitness.values
    
    # and apply mutation
    for ind in offspring:
        for tree, pset in zip(ind, psets):
            if random.random() < MUTPB:
                toolbox.mutate(individual=tree, pset=pset)
                del ind.fitness.values
                
    # evaluate individuals with invalid fitness
    invalids = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalids:
        ind.fitness.values = toolbox.evaluate(ind)
        
    pop = offspring
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals = len(invalids), **record)
    print(logbook.stream)
    
print("Best individual: ", hof[0][0], hof[0].fitness)

gen	evals	std     	min	avg     	max
0  	4000 	0.285019	-10	-7.99625	-6 
1  	3894 	0.318704	-10	-7.94825	-6 
2  	3878 	0.369755	-10	-7.909  	-6 
3  	3858 	0.439886	-10	-7.85   	-6 
4  	3878 	0.50636 	-10	-7.76   	-6 
5  	3884 	0.54726 	-10	-7.6975 	-6 
6  	3896 	0.605541	-10	-7.62775	-5 
7  	3856 	0.641251	-10	-7.51725	-5 
8  	3894 	0.690051	-10	-7.427  	-5 
9  	3886 	0.718928	-10	-7.33525	-5 
10 	3904 	0.759253	-10	-7.22975	-5 
11 	3874 	0.797908	-10	-7.14175	-4 
12 	3896 	0.835401	-10	-7.02925	-4 
13 	3902 	0.869707	-11	-6.90325	-4 
14 	3898 	0.891427	-12	-6.79725	-3 
15 	3902 	0.928822	-10	-6.71975	-3 
16 	3892 	0.963523	-10	-6.63275	-3 
17 	3880 	1.00214 	-12	-6.53475	-3 
18 	3894 	1.06424 	-12	-6.43   	-3 
19 	3880 	1.09421 	-10	-6.28275	-2 
20 	3868 	1.16171 	-13	-6.12325	-2 
21 	3892 	1.19513 	-11	-6.0125 	-2 
22 	3906 	1.26948 	-12	-5.88525	-2 
23 	3868 	1.2919  	-12	-5.728  	-2 
24 	3892 	1.35261 	-12	-5.59725	-2 
25 	3888 	1.42599 	-12	-5.46075	-2 
26 	3892 	1.37294 	-12	-5.25

In [17]:
print("ADF1")
print(hof[0][0])
print("ADF0")
print(hof[0][1])
print("MAIN")
print(hof[0][2])

ADF1
NAND(or_(ADF1(NAND(ARG1, ARG0), ADF1(and_(ARG3, ADF0(and_(NAND(ARG3, ARG2), NAND(ARG0, ARG3)), or_(ARG0, ARG1))), ARG2)), NOR(ARG3, NAND(and_(or_(ARG3, and_(ARG1, ARG3)), and_(ARG3, ARG3)), ADF1(NAND(ARG1, ARG1), NAND(ARG3, ARG0))))), NAND(NAND(ARG3, ARG3), ARG3))
ADF0
and_(or_(ARG0, ARG1), ARG1)
MAIN
or_(NOR(or_(ARG0, ARG1), ARG0), and_(ARG0, or_(ARG1, ARG1)))


In [18]:
f = toolbox.compile(expr=hof[0])

for in_ in inputs:
    print(f(*in_))
    
print(outputs)

False
False
True
True
False
True
True
False
False
True
True
False
True
False
False
True
[True, False, False, True, False, True, True, False, False, True, True, False, True, False, False, True]
